<a href="https://colab.research.google.com/github/ananyakbcodes/TEAM_PROJECT/blob/Cassiel/3_train_weather_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import pandas as pd
import time
import os
from sklearn.ensemble import RandomForestClassifier
import joblib

# API key
API_KEY = "38b9a16faaae2ad527c03a324121272c"

# Cities to collect data from
CITIES = ["Panaji", "Mumbai", "Delhi", "Chennai", "Kolkata", "Pune"]

# Storage lists
rows = []

# Create folders if not exist
os.makedirs("/content/drive/MyDrive/crop_project/data", exist_ok=True)
os.makedirs("/content/drive/MyDrive/crop_project/models", exist_ok=True)

# Collect weather data
for city in CITIES:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city},IN&appid={API_KEY}&units=metric"
    r = requests.get(url).json()

    if "main" in r:
        temp = r["main"]["temp"]
        humidity = r["main"]["humidity"]
        pressure = r["main"]["pressure"]
        wind_speed = r["wind"]["speed"]
        rainfall = r.get("rain", {"1h": 0}).get("1h", 0)
        label = 1 if humidity > 80 and 20 <= temp <= 35 and rainfall > 2 else 0
        rows.append([city, temp, humidity, rainfall, pressure, wind_speed, label])
        print(f"Collected: {city}")
    else:
        print(f"Skipped {city}: {r.get('message', 'Unknown error')}")
        print("Full response:", r)

    time.sleep(1)

# Create DataFrame and save it
df = pd.DataFrame(rows, columns=["city", "temp", "humidity", "rainfall", "pressure", "wind_speed", "label"])
df.to_csv("/content/drive/MyDrive/crop_project/data/weather_dataset.csv", index=False)
print("Saved dataset to weather_dataset.csv")

# Train and save RandomForest model
X = df[["temp","humidity","rainfall","pressure","wind_speed"]]
y = df["label"]

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(
    {"model": model, "features": X.columns.tolist()},
    "/content/drive/MyDrive/crop_project/models/weather_model.pkl"
)

print("Model trained and saved to weather_model.pkl")

# Show preview
df.head()


Collected: Panaji
Collected: Mumbai
Collected: Delhi
Collected: Chennai
Collected: Kolkata
Collected: Pune
Saved dataset to weather_dataset.csv
Model trained and saved to weather_model.pkl


,city,temp,humidity,rainfall,pressure,wind_speed,label
0,Panaji,27.00,78,0,1013,3.09,0
1,Mumbai,26.99,89,0,1012,4.63,0
2,Delhi,25.05,50,0,1010,2.06,0
3,Chennai,30.73,76,0,1010,4.02,0
4,Kolkata,25.97,78,0,1009,2.57,0


weather_dataset.csv
weather_model.pkl
